<a href="https://colab.research.google.com/github/jbottum/t5pat/blob/main/score1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install einops
!pip install transformers
!pip install --upgrade transformers
!pip install accelerate
!pip install torch
!pip install langchain
!pip install bert-score

In [2]:
import time
import matplotlib.pyplot as plt
from langchain.llms import HuggingFacePipeline
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, pipeline
import transformers
import torch

import bert_score
import logging
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)


import os
# Disable parallelism and avoid the warning message
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
user_id = 'jbottum'
project_id = 'project1'
model_id = 'google/flan-t5-large'

# Define prompts and types
prompts = [
    'What is the capital of Germany?',
    'What is the capital of Spain?',
    'What is the capital of Canada?',
    'What is the next number in the sequence: 2, 4, 6, 8, ...? If all cats have tails, and Fluffy is a cat, does Fluffy have a tail?',
    'If you eat too much junk food, what will happen to your health? How does smoking affect the risk of lung cancer?',
    'In the same way that pen is related to paper, what is fork related to? If tree is related to forest, what is brick related to?',
    'Every time John eats peanuts, he gets a rash. Does John have a peanut allergy? Every time Sarah studies for a test, she gets an A. Will Sarah get an A on the next test if she studies?',
    'All dogs have fur. Max is a dog. Does Max have fur? If it is raining outside, and Mary does not like to get wet, will Mary take an umbrella?',
    'If I had studied harder, would I have passed the exam? What would have happened if Thomas Edison had not invented the light bulb?',
    'The center of Tropical Storm Arlene, at 02/1800 UTC, is near 26.7N 86.2W. This position is about 425 km/230 nm to the west of Fort Myers in Florida, and it is about 550 km/297 nm to the NNW of the western tip of Cuba. The tropical storm is moving southward, or 175 degrees, 4 knots. The estimated minimum central pressure is 1002 mb. The maximum sustained wind speeds are 35 knots with gusts to 45 knots. The sea heights that are close to the tropical storm are ranging from 6 feet to a maximum of 10 feet.  Precipitation: scattered to numerous moderate is within 180 nm of the center in the NE quadrant. Isolated moderate is from 25N to 27N between 80W and 84W, including parts of south Florida.  Broad surface low pressure extends from the area of the tropical storm, through the Yucatan Channel, into the NW part of the Caribbean Sea.   Where and when will the storm make landfall?',
]

types = [
    'Knowledge Retrieval',
    'Knowledge Retrieval',
    'Knowledge Retrieval',
    'Logical Reasoning',
    'Cause and Effect',
    'Analogical Reasoning',
    'Inductive Reasoning',
    'Deductive Reasoning',
    'Counterfactual Reasoning',
    'In Context'
]

# Example reference answers
reference_list = [
    'The capital of Germany is Berlin',
    'The capital of Spain is Madrid',
    'The capital of Canada is Ottawa',
    'The next number in the sequence is 10.  Yes, Fluffy is a cat and therefore has a tail.',
    'Eating junk food can result in health problems like weight gain and high cholesterol. Smoking can cause lung issues including cancer.',
    'Fork is related to a plate.  A brick is related to a building.',
    'Maybe, to determine if Johns rash is caused by peanuts, he should take an allergy test for peanuts.   Maybe, Sarah will likely do well if she studies and she may be able to get an A.',
    'Yes, Max is a dog and has fur.   Yes, Mary will take an umbrella.',
    'Yes, if you studied harder, you would have passed the test.  If Thomas Edison did not invent the light blub, another inventor would have created the light bulb.',
    'If Arlene continues in the same direciton and speed, storm will make landfall in the Forida Keys in 18 hours from this report.'
]



In [4]:
data = {
    "user_id": "jbottum",
    "project_id": "project1",
    "model_id": "google/flan-t5-large",
    "prompts": prompts,
    "types": types,
    "reference_list": reference_list
}


In [5]:
import json

# Save data as JSON
with open("data.json", "w") as json_file:
    json.dump(data, json_file, indent=4)


In [6]:
import csv

# Transpose the data to convert it to CSV format
transposed_data = list(zip(prompts, types, reference_list))

# Save data as CSV
with open("data.csv", "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Prompts", "Types", "Reference List"])
    writer.writerows(transposed_data)


JSON Schema for the Data:

{
  "user_id": "string",
  "project_id": "string",
  "model_id": "string",
  "prompts": [
    "string",
    ...
  ],
  "types": [
    "string",
    ...
  ],
  "reference_list": [
    "string",
    ...
  ]
}


Explanation:

user_id (string): The user_id field is a string that should contain the unique identifier of the user who owns the project. This identifier helps in associating the data with a specific user account or project owner.

project_id (string): The project_id field is a string that should contain the unique identifier of the project associated with the data. It allows users to differentiate between various projects and their corresponding data.

model_id (string): The model_id field is a string that should contain the identifier of the model used for the project. This identifier helps users identify the specific language model or AI model used for processing the prompts.

prompts (list of strings): The prompts field is a list of strings. Each string in this list represents a different type of question or scenario that the model is expected to answer or process. Users should provide the prompts in the order they want them processed.

types (list of strings): The types field is a list of strings. Each string in this list corresponds to the type of the prompt in the same position in the prompts list. The types categorize the prompts based on their nature, such as "Knowledge Retrieval," "Logical Reasoning," etc.

reference_list (list of strings): The reference_list field is a list of strings. Each string in this list corresponds to the example reference answer for the prompt in the same position in the prompts list. These reference answers represent the expected model responses for each prompt.

Note:

The ... in the schema represents that the respective lists (prompts, types, and reference_list) can contain an arbitrary number of elements, depending on the number of prompts and their corresponding information.
Users building JSON files should follow the exact structure and field names as specified in the schema to ensure compatibility and proper data representation within the app.

CSV Schema for the Data:

Prompts,Types,Reference List
string,string,string
...


Explanation:

Prompts (string): This column should contain the prompts that represent different types of questions or scenarios for the model to answer or process.
Types (string): This column should contain the types corresponding to each prompt. These types categorize the prompts based on the nature of the question or scenario, such as "Knowledge Retrieval," "Logical Reasoning," etc.
Reference List (string): This column should contain the example reference answers for each prompt. These reference answers represent the expected model responses to the corresponding prompts.
Note:

The ... in the schema indicates that there can be an arbitrary number of rows in the CSV file, each corresponding to a different prompt, type, and reference answer.
Users building CSV files should strictly follow the order of columns as specified in the schema to ensure proper mapping and accuracy. Each row in the CSV should contain data for one prompt, its corresponding type, and the reference answer.

In [7]:
import json

# Load data from the JSON file into a dictionary
with open("data.json", "r") as json_file:
    data = json.load(json_file)


In [8]:
user_id = data["user_id"]
project_id = data["project_id"]
model_id = data["model_id"]
prompts = data["prompts"]
types = data["types"]
reference_list = data["reference_list"]


In [9]:
import json

# Load data from the JSON file into a dictionary
with open("data.json", "r") as json_file:
    data = json.load(json_file)

# Print the dictionary to check the loaded data
print(data)


{'user_id': 'jbottum', 'project_id': 'project1', 'model_id': 'google/flan-t5-large', 'prompts': ['What is the capital of Germany?', 'What is the capital of Spain?', 'What is the capital of Canada?', 'What is the next number in the sequence: 2, 4, 6, 8, ...? If all cats have tails, and Fluffy is a cat, does Fluffy have a tail?', 'If you eat too much junk food, what will happen to your health? How does smoking affect the risk of lung cancer?', 'In the same way that pen is related to paper, what is fork related to? If tree is related to forest, what is brick related to?', 'Every time John eats peanuts, he gets a rash. Does John have a peanut allergy? Every time Sarah studies for a test, she gets an A. Will Sarah get an A on the next test if she studies?', 'All dogs have fur. Max is a dog. Does Max have fur? If it is raining outside, and Mary does not like to get wet, will Mary take an umbrella?', 'If I had studied harder, would I have passed the exam? What would have happened if Thomas Ed

In [10]:
# Create empty lists to store generation times, model load times, tokenizer load times, and pipeline load times
generation_times = []
model_load_times = []
tokenizer_load_times = []
pipeline_load_times = []

prompt_types = []

In [11]:
import time
import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, pipeline, HuggingFacePipeline

# Load tokenizer
tokenizer_start_time = time.time()
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer_end_time = time.time()

# Load model
model_start_time = time.time()
if model_id == 'tiiuae/falcon-7b':
    model = AutoModelForCausalLM.from_pretrained(model_id)
else:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
model_end_time = time.time()

# Load pipeline
pipe_start_time = time.time()
if model_id == 'tiiuae/falcon-7b':
    pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        device_map="auto",
    )
else:
    pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)
    local_llm = HuggingFacePipeline(pipeline=pipe)
pipe_end_time = time.time()

# Store loading times
model_load_times.append(model_end_time - model_start_time)
tokenizer_load_times.append(tokenizer_end_time - tokenizer_start_time)
pipeline_load_times.append(pipe_end_time - pipe_start_time)

In [14]:
import time
import torch
from transformers import AutoModelForSeq2SeqLM, pipeline
import bert_score

# Print model results
print()
print(f"Results for model: {model_id}")
print("=" * 30)

# Knowledge retrieval examples (for 'tiiuae/falcon-7b')
if model_id == 'tiiuae/falcon-7b':
    sequences = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        device_map="auto",
        task="Draft an apology email to a customer who experienced poor service.",
        num_return_sequences=3,
        max_length=100,
        min_length=10,
    )
    for i, seq in enumerate(sequences):
        print(f"Retrieval Example {i + 1}:")
        print(seq["generated_text"])
        print("=" * 10)

# Generation examples (for other models)
else:
    pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=50)
    for i, prompt in enumerate(prompts):
        start_time = time.time()
        generated = pipe(prompt, num_return_sequences=1)[0]['generated_text']
        end_time = time.time()
        generation_time = end_time - start_time

        # Store generation times
        generation_times.append(generation_time)

        # Print generation example
        print(f"Generation Example {i + 1}:")
        print("Prompt:", prompt)
        print("Generated Text:", generated)

        prompt_types.append(types[i])

        candidates = [generated]  # Wrap generated text in a list to handle multiple candidates

        for j, candidate in enumerate(candidates):
            # Calculate BERTScore
            reference = reference_list[i]
            print(f"Reference Answer:", reference)

            P, R, F1 = bert_score.score([candidate], [reference], lang="en", verbose=False)
            print("Precision:", P.numpy())
            print("Recall:", R.numpy())
            print("F1 Score:", F1.numpy())

        print("=" * 10)



Results for model: google/flan-t5-large
Generation Example 1:
Prompt: What is the capital of Germany?
Generated Text: berlin
Reference Answer: The capital of Germany is Berlin


Precision: [0.8048427]
Recall: [0.82158786]
F1 Score: [0.81312907]
Generation Example 2:
Prompt: What is the capital of Spain?
Generated Text: turin
Reference Answer: The capital of Spain is Madrid
Precision: [0.76088876]
Recall: [0.7964282]
F1 Score: [0.77825296]
Generation Example 3:
Prompt: What is the capital of Canada?
Generated Text: toronto
Reference Answer: The capital of Canada is Ottawa
Precision: [0.77873576]
Recall: [0.78586817]
F1 Score: [0.78228575]
Generation Example 4:
Prompt: What is the next number in the sequence: 2, 4, 6, 8, ...? If all cats have tails, and Fluffy is a cat, does Fluffy have a tail?
Generated Text: yes
Reference Answer: The next number in the sequence is 10.  Yes, Fluffy is a cat and therefore has a tail.
Precision: [0.8003874]
Recall: [0.808023]
F1 Score: [0.80418706]
Generation Example 5:
Prompt: If you eat too much junk food, what will happen to your health? How does smoking affect the risk of lung cancer?
Generated Text: no
Reference Answer: Eati